Starting with reading the datasets created for performance analysis in inception V3.ipynb notebook 

In [ ]:
import h5py
hdf5File = h5py.File(r'..\..\data\processed\predictions.hdf5', 'r')
list(hdf5File.keys())

Nice! let's read the topPred2D dataset that contains the Top-1, 10-Crop image prediction

In [ ]:
Prediction2D = hdf5File['topPred2D']
Prediction2D[:10]

In [ ]:
%%time
from tqdm import tqdm
right_counter = 0
for i in tqdm(range(len(Prediction2D))):
    if Prediction2D[i][0] == Prediction2D[i][11]:
        right_counter += 1
        
print('Top-1 Accuracy, 10-Crop: {0:.2f}%'.format(right_counter / len(Prediction2D) * 100))

We are getting 78.72% accuracy in first trial. Now let's see the Top-5 accuracy, 10 crops. We are assuming that both topPred2D top5Pred3d contains prediction in same order for source image.

In [ ]:
Prediction3DTop5 = hdf5File['top5Pred3D']
Prediction3DTop5[:2]

In [ ]:
%%time
import numpy as np
topPred2D = np.zeros(5)
top_5_counter = 0

for i in tqdm(range(len(Prediction3DTop5))):
    for j in range(5):
        counts = np.bincount(list(np.transpose(Prediction3DTop5[i])[j]))
        mostCommonPrediction = np.argmax(counts)
        topPred2D[j] =  mostCommonPrediction
        
    if (Prediction2D[i][0] in topPred2D):
        top_5_counter += 1
        
print('Top-5 Accuracy, 10-Crop: {0:.2f}%'.format(top_5_counter / len(Prediction2D) * 100))

For Top-5 accuracy we get 87.28%. Which is a big jump. For calculating the value we used the most common value among same index of top-5 prediction for each crop. We can use another technique where we use all the values in top-5 and find the top 5 most common values of it. 

In [ ]:
%%time
from collections import Counter
topPred2D = np.zeros(5)
top_5_counter = 0

for i in tqdm(range(len(Prediction3DTop5))):
    for j in range(5):
        mostCommonFlattenedList = Counter(list(Prediction3DTop5[i].flatten())).most_common(5)
        topPred2D[j] =  mostCommonFlattenedList[j][0]
      
    if (Prediction2D[i][0] in topPred2D):
        top_5_counter += 1
        
print('Top-5 Accuracy, 10-Crop: {0:.2f}%'.format(top_5_counter / len(Prediction2D) * 100))

Nice!! Got a jump of more than 6% accuracy. 

### Result Visualization

More than anything, at this point I am interested in ranking classes based on their accuracies.

In [ ]:
accuracyList = [] # list to contain individual accuracies

for item in tqdm(range(101)):
    subGroup = Prediction2D[Prediction2D[:,0]==item,-1] # subGrouping data for each item
    count = 0
    for predIndex in range(len(subGroup)):
        if(subGroup[predIndex] == item):
            count += 1
            
    accuracy = count/len(subGroup)
    accuracyList.append((item, accuracy))
    
accuracyList[:10]

Cool! so we have individual accuracies. let's see which index has the worst accuracy of all.

In [ ]:
[item for item in accuracyList if min(accuracyList)[1] in item]

This means that Apple Pie and Hummus had the worst accuracy, Checking the best accuracy.

In [ ]:
[item for item in accuracyList if max(accuracyList)[1] in item]

Chicken quesadilla, Croque madame, and Waffles had the best accuracy. 